In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

# Load data
df = pd.read_csv(r"D:\processed-data.csv")

# Drop unnecessary columns using column indexing and rename the target column
df = df.drop(df.columns[[4, 5]], axis=1)  # Drop columns by index
df.rename(columns={'Severity_None': 'Target'}, inplace=True)

# Prepare data for training and testing
x = df.drop(columns=['Target'])  # Features
y = df['Target']  # Target variable: 'Target' column represents severity levels

# Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Initialize the Gradient Boosting Classifier with adjusted hyperparameters
model = GradientBoostingClassifier(
    n_estimators=100,  # Increase the number of estimators
    learning_rate=0.1,  # Adjust the learning rate
    max_depth=5,  # Increase the maximum depth of the trees
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
)

# Train the model
model.fit(x_train, y_train)


In [ ]:
# Function to map 'yes' or 'no' inputs to integers
def map_yes_no_to_int(answer):
    if answer.lower() == 'yes':
        return 1
    elif answer.lower() == 'no':
        return 0
    else:
        raise ValueError("Invalid input! Please enter 'yes' or 'no'.")


In [ ]:
# Function to check threshold based on symptoms, age, and gender
def check_threshold(symptoms, age, gender):
    if all(symptoms) and age >= 60:
        return True
    elif all(symptoms) and age < 11:
        return True
    elif all(symptoms):
        return True
    elif not any(symptoms) and gender == 'male' and age >= 25:
        return False
    else:
        return False


In [ ]:
# Function to get recommendation based on severity prediction
def get_recommendation(severity):
    if severity == 0:
        return 'Your asthma condition is currently under control. Continue to monitor your symptoms regularly.'
    elif severity == 1 or severity == 2:
        return '''You are experiencing mild to moderate asthma symptoms. Try some home remedies such as:
            1. Steam Inhalation: Inhale steam from hot water to open up the airways.
            2. Staying Hydrated: Drink plenty of water to keep the airways moist.
            3. Using a Humidifier: Add moisture to the air with a humidifier to prevent dryness in the airways.
            4. Breathing Exercises: Practice deep breathing exercises and pursed-lip breathing to improve lung function.
            5. Avoiding Triggers: Identify and avoid triggers such as smoke, dust, pollen, and pet dander.
            6. Maintaining a Clean Environment: Keep the home clean and free of dust, mold, and allergens.'''
    elif severity == 3:
        return '''You are experiencing severe asthma symptoms. Please seek immediate medical attention. In the meantime, you may find the following resources helpful:
            1. [How to ease asthma symptoms - 3 effective breathing exercises by Airofit](https://www.youtube.com/watch?v=FyjZLPmZ534)
            2. [Exercise-Induced Asthma by CNN](https://www.youtube.com/watch?v=B8pNeYFZNew)
            3. [ASTHMA / how to cure exercise induced wheezing naturally by Andrew Folts](https://www.youtube.com/watch?v=jv-revgQdPE)
            4. [Easy tips to treat Asthma & Bronchitis | Dr. Hansaji Yogendra by The Yoga Institute](https://www.youtube.com/watch?v=JwRG8AsStLQ)
            5. [Breathing Exercises for COPD, Asthma, Bronchitis & Emphysema - Ask Doctor Jo by AskDoctorJo](https://www.youtube.com/watch?v=dpTNUGwXbTU)'''


In [ ]:
# Function to perform prediction
def predict_asthma(name, tiredness, dry_cough, difficulty_breathing, sore_throat, nasal_congestion, runny_nose, age, gender):
    try:
        tiredness = map_yes_no_to_int(tiredness)
        dry_cough = map_yes_no_to_int(dry_cough)
        difficulty_breathing = map_yes_no_to_int(difficulty_breathing)
        sore_throat = map_yes_no_to_int(sore_throat)
        nasal_congestion = map_yes_no_to_int(nasal_congestion)
        runny_nose = map_yes_no_to_int(runny_nose)
        age = int(age)

        # Check if all symptoms are 'no'
        if tiredness == dry_cough == difficulty_breathing == sore_throat == nasal_congestion == runny_nose == 0:
            severity_prediction = 0
        else:
            symptoms = [tiredness, dry_cough, difficulty_breathing, sore_throat, nasal_congestion, runny_nose]
            if check_threshold(symptoms, age, gender):
                severity_prediction = 3  # Set severity to maximum if threshold is met
            else:
                none_experiencing = 1 if sum(symptoms) == 0 else 0
                user_input = [tiredness, dry_cough, difficulty_breathing, sore_throat, nasal_congestion, runny_nose,
                              none_experiencing,  # None_Experiencing feature
                              1 if age <= 9 else 0,  # Age_0-9 feature
                              1 if age >= 10 and age <= 19 else 0,  # Age_10-19 feature
                              1 if age >= 20 and age <= 24 else 0,  # Age_20-24 feature
                              1 if age >= 25 and age <= 59 else 0,  # Age_25-59 feature
                              1 if age >= 60 else 0,  # Age_60+ feature
                              1 if gender == 'female' else 0,  # Gender_Female feature
                              1 if gender == 'male' else 0,  # Gender_Male feature
                              0,  # Severity_Mild placeholder
                              0]  # Severity_Moderate placeholder
                severity_prediction = model.predict([user_input])[0]

        recommendation = get_recommendation(severity_prediction)

        return f"Hello {name}!\nSeverity Prediction: {severity_prediction}\nRecommendation: {recommendation}\nGender: {gender}"

    except Exception as e:
        return f"An error occurred: {str(e)}"
